In [ ]:
import numpy as np
import pandas as pd
import pyspark as ps
import matplotlib.pyplot as plt

Here we preform an exploration of the partial Spinn3r dataset provided.

We first find how to explore the data using one sampel json file:

In [ ]:
EXAMPLE_PATH = 'swiss-tweet/example.json'

In [ ]:
example = pd.read_json(EXAMPLE_PATH)
example.columns

We note that the data we are interessted in is contained in the _source variable, in object format:

In [ ]:
example.dtypes

The object format is a nested json that we could not find how to extract dirrectly using the read json funtion provided. We thus use a json normaliter contained in the pandas libary to extract it:

In [ ]:
from pandas.io.json import json_normalize
import json

We set max dispaly columns to a larger number in able to see all columns:

In [ ]:
pd.set_option('display.max_columns', 58)
example1 = json_normalize(example._source[0])
print(example1.columns)
print(len(example1.columns))

In [ ]:
print(example1)

From the spinner documentation we see that:

The values we are most interessted in:
 - main: content of tweet
 - sentiment: overal sentiment of tweet
 - author_gender: gender of author
 - source_location: to know where was posted
 - lang: to get the language of the tweet
 - source_created: to know when it was made


This brings us the the first things we need to be careful about:
 - what does the sentiment realy say about the tweet?
 - is the location accurate?

Additionaly, after some trying around we find that the inbuilt read_json function was not ideal for or pourposed. We thus open the file using 'open' and transform it into a dataframe using json_normalize. 

In [ ]:
with open(EXAMPLE_PATH
         ) as data_file:    
    data = json.load(data_file)

In [ ]:
#example of how data looks like, for first tweet
data[0]['_source']

In [ ]:
twitter_df = json_normalize(data)

In [ ]:
#nicer naming
twitter_df.columns = [ column.replace('_source.','') for column in twitter_df.columns]
print(twitter_df.columns)
print(twitter_df.dtypes)

In [ ]:
twitter_df.head()

In [ ]:
language_freq = twitter_df['lang'].value_counts()
language_freq

In [ ]:
gender_freq = twitter_df['author_gender'].value_counts()
gender_freq

In [ ]:
sentiment_freq = twitter_df['sentiment'].value_counts()
sentiment_freq

In [ ]:
negative_tweets = twitter_df[twitter_df['sentiment'] == 'NEGATIVE']['main'] 
negative_tweets.map(lambda x: ':(' in x).value_counts() ##so it's just the smileys

In [ ]:
negative_tweets

In [ ]:
positive_tweets = twitter_df[twitter_df['sentiment'] == 'POSITIVE']['main'] 
positive_tweets

In [ ]:
positive_tweets.map(lambda x: (':-)' in x) or (':)' in x) or (';)' in x)).value_counts() #also just smileys basicaly

In [ ]:
plt.hist(twitter_df.main_length, bins=50)
plt.show()

We can see right away that the location names depend on the language of the user.

In [ ]:
twitter_df['source_location'].head()

In [ ]:
twitter_df['source_location'].value_counts()

We also dirrectly see that often, the location is not very accurate, giving us only 'switzerland'.
This will not be enough for our analysis as we want to see if we can find a difference between the röstigraben.
But we could still work with this by using language as an inicator instead.

In [ ]:
twitter_df['source_spam_probability'].value_counts() #none of them are suspected to be spam

In [ ]:
twitter_df['tags'].dropna()

We see a münchen in there -> munich -> this indicated that geolocation may be off

In [ ]:
twitter_df['published'] = pd.to_datetime(twitter_df['published'])

In [ ]:
twitter_df['published'].map(lambda x: (x.day, x.month)).value_counts() #all published on same day, same month

In [ ]:
twitter_df['source_user_interactions'].value_counts()

In [ ]:
#twitter_df['source_following'].value_counts() just to see, not that usefull as values, make graph maybe if useable.

In [ ]:
#twitter_df['source_favorites'].value_counts()

In [ ]:
#twitter_df['source_followers'].value_counts()

In [ ]:
twitter_df['source_likes'].value_counts() #no facebook likes